# 1a. Electrolyser Characteristics



```{note}
This example uses plotly == 5.6.0 to plot results.
```

Importing Packages

In [2]:
# NEMGLO Packages
from nemglo import *

# Generic Packages
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
# Display plotly chart in a browser (optional)
import plotly.io as pio
pio.renderers.default = "browser"

Loading the Historical AEMO price data

In [4]:
inputdata = nemosis_data(intlength=30, local_cache=r'E:\TEMPCACHE')
start = "02/01/2020 00:00"
end = "09/01/2020 00:00"
region = 'VIC1'
inputdata.set_dates(start, end)
inputdata.set_region(region)
prices = inputdata.get_prices()

Compiling data for table DISPATCHPRICE.
Returning DISPATCHPRICE.


## Hydrogen Production Benefit Price
The **H2 price** feature is optional, yet recommended if your model does not use production targets, in order to incentivise the electrolyser to operate and maximise its load capacity factor. The `h2_price_kg` parameter is set withint the `Electrolyser.load_h2_parameters_preset` function call.

This example demonstrates the impact of changing this parameter on the optimiser results.

In [5]:
P2G = Plan(identifier = "P2G")
P2G.load_market_prices(prices)

In [6]:
h2_price_points = [1.0,2.0,3.0,4.0,5.0,6.0]

In [7]:
result_load = []
for h2_price in h2_price_points:
    P2G = Plan(identifier = "P2G")
    P2G.load_market_prices(prices)

    h2e = Electrolyser(P2G, identifier='H2E')
    h2e.load_h2_parameters_preset(capacity = 100.0,
                                maxload = 100.0,
                                minload = 20.0,
                                offload = 0.0,
                                electrolyser_type = 'PEM',
                                sec_profile = 'fixed',
                                h2_price_kg = h2_price)
    h2e.add_electrolyser_operation()

    P2G.optimise()

    result_load += [P2G.get_load()]


OPTIMISATION COMPLETE, Obj Value: -52760.22252020205
OPTIMISATION COMPLETE, Obj Value: -80327.85879292933
OPTIMISATION COMPLETE, Obj Value: -137818.77469823236
OPTIMISATION COMPLETE, Obj Value: -273552.0548232324
OPTIMISATION COMPLETE, Obj Value: -478009.83247474744
OPTIMISATION COMPLETE, Obj Value: -719478.0522853544


In [37]:
PALETTE = ['#b2d4ee','#849db1','#4f6980',
           '#B4E3BC','#89AE8F','#638b66',
           '#ffb04f','#de9945','#af7635',
           '#ff7371','#d6635f','#b65551',
           '#AD134C','#cc688d','#ff82b0']  

fig = make_subplots(rows=8, cols=1, subplot_titles=("<b>Price</b>",None,"<b>Outputs</b>"),
                    specs=[[{'rowspan':2}], [{}], [{}], [{}], [{}], [{}], [{}], [{}]])
fig.update_annotations(font=dict(size=20, family="Times New Roman"))

fmt_timestamps = [dt.strftime(prices['Time'][i], "%d-%b %H:%M") \
                  for i in range(len(prices))]
fig.add_trace(go.Scatter(x=prices['Time'], y=prices['Prices'], line={'color':'#972f42'},
                         showlegend=False), row=1, col=1)
fig.update_yaxes(title="Price<br>($/MWh)", showgrid=False, gridcolor='slategrey',
                 range=[-250,150], mirror=True, titlefont=dict(size=18),
                 tickfont=dict(size=18), overlaying="y", side="left", row=1, col=1)
fig.update_xaxes(title=None, mirror=True, showticklabels=False, row=1, col=1)

for idx, element in enumerate(result_load):
    if idx < 5:
        fig.add_trace(go.Scatter(x=element['time'], y=element['value'],name="${}/kg" \
            .format(h2_price_points[idx]), line={'color':PALETTE[1+3*idx]}, xaxis="x2",
                    yaxis="y"), row=idx+3, col=1)
        fig.update_yaxes(title="Load<br>(MW)", showgrid=False, range=[-10,140],
                         mirror=True, titlefont=dict(size=18), tickfont=dict(size=18),
                         tickvals=[i for i in range(0, 151, 50)], row=idx+3, col=1)
        fig.update_xaxes(showticklabels=False, row=idx+3, col=1)
    else:
        fig.add_trace(go.Scatter(x=element['time'], y=element['value'],name="${}/kg" \
            .format(h2_price_points[idx]), line={'color':'#7F22A6'}), row=idx+3, col=1)
        fig.update_yaxes(title="Load<br>(MW)", showgrid=False, range=[-10,140],
                         mirror=True, titlefont=dict(size=18), tickfont=dict(size=18),
                         tickvals=[i for i in range(0, 151, 50)], row=idx+3, col=1)
        fig.update_xaxes(title="Time", showgrid=False, mirror=False,
                         titlefont=dict(size=18), tickfont=dict(size=18), tickangle=-45,
                         tickformat="%d-%b",  domain=[0, 1], row=idx+3, col=1)

fig.update_layout(title='<b>NEMGLO Electrolyser H2 Price Sensitivity<br>' + \
                        '<sup>VIC: Jan-2020</sup></b>', titlefont=dict(size=24),
                  margin=dict(l=20, r=20, t=100, b=0), 
                  legend=dict(xanchor='center',x=0.5, y=-0.15, orientation='h',
                              font=dict(size=20)),
                  template="simple_white", font_family="Times New Roman",
                  width=1000, height=800)

for ser in fig['data']:
  ser['text'] = [dt.strftime(prices['Time'][i], "%d-%b %H:%M") \
                 for i in range(len(prices))]
  ser['hovertemplate'] = 'Time: %{text}<br>Value: %{y}'

fig.show()

```{include} example_electrolyser_h2price.html 
```

## Minimum Stable Loading
The MSL feature is, by default, considered in NEMGLO by `load_h2_parameters_preset` and `add_electrolyser_operation` if the `minload` value parsed is greater than zero.

Creating the `Plan` and `Electrolyser` objects

In [228]:
P2G = Plan(identifier = "P2G")
P2G.load_market_prices(prices)

In [229]:
h2e = Electrolyser(P2G, identifier='H2E')

In [230]:
h2e.load_h2_parameters_preset(capacity = 100.0,
                              maxload = 100.0,
                              minload = 20.0,
                              offload = 0.0,
                              electrolyser_type = 'PEM',
                              sec_profile = 'fixed',
                              h2_price_kg = 6.0)

In [231]:
h2e.add_electrolyser_operation()

In this example, we use additional features of `set_production_target` to force the electrolyser to operate closer to its minimum stable load. There is still a price incentive to maximise production, that is `h2_price_kg` is set to $6/kg above, yet we add a limit maximum production limit of 15 tonnes of H2 per day and a minimum production limit of 200 kg per hour.

```{note} The <code>_set_</code> function calls are more advanced features of NEMGLO which are **post-operands**. They must be called after <code>add_electrolyser_operation</code> since the <code>add_</code> function creates optimiser variables which are later used by <code>set_</code> functions to create additional constraints
```

In [232]:
h2e._set_production_target(target_value=15000,bound="max", period="day")
h2e._set_production_target(target_value=200,bound="min", period="hour")

Additionally, we will impose a cost on the ramping of the electrolyser load to mimick more realistic behaviour. This 'smooths' the load profile which otherwise looks very 'jaggered' as a result of the optimiser setting load in one interval zero and high in the next in order to meet the production target. 

Such a cost can be treated as a shadow cost for the optimiser. It would not be incurred by the load participant. 

In [233]:
h2e._set_ramp_variable()
h2e._set_ramp_cost(cost=10)

Now lets run the optimiser, extract results and plot them. Here we can also show the H2 produced to verify that the production targets are met, simply by using `get_production`.

In [234]:
P2G.optimise(solver_name="CBC", save_debug=False, save_results=False, results_dir=None)

OPTIMISATION COMPLETE, Obj Value: -362357.14000366995


In [235]:
result_load = P2G.get_load()
result_product = P2G.get_production()

In [236]:
fig = go.Figure()
fig.update_layout(title='<b>NEMGLO Electrolyser MSL Demonstration<br><sup>VIC: Jan-2020</sup></b>', titlefont=dict(size=24),
                  margin=dict(l=20, r=20, t=60, b=0),
                  xaxis=dict(title="Time", showgrid=False, mirror=True, titlefont=dict(size=18), \
                    tickfont=dict(size=18), tickangle=-45, tickformat="%d-%b",  domain=[0.15, 1]),
                  yaxis=dict(title="Load Dispatch (MW)", showgrid=False, range=[-10,140], mirror=True, titlefont=dict(size=18),\
                    tickfont=dict(size=18), tickvals=[i for i in range(-20, 140, 20)], rangemode="tozero", scaleanchor="y2", scaleratio=4, color='#7F22A6'),
                  yaxis2=dict(title="H2 volume (kg)", showgrid=False, mirror=True, titlefont=dict(size=18),\
                    tickfont=dict(size=18), anchor="free", overlaying="y", side="left", position=0.02, rangemode="tozero", \
                      scaleanchor="y", scaleratio=1, color="darkorange"),
                  yaxis3=dict(title="Price ($/MWh)", showgrid=False, gridcolor='slategrey', range=[-250,150], mirror=True, \
                    titlefont=dict(size=18),tickfont=dict(size=18), anchor="x", overlaying="y", side="right", color="#972f42"),
                  legend=dict(xanchor='center',x=0.55, y=-0.35, orientation='h', font=dict(size=20)),
                  template="simple_white",
                  font_family="Times New Roman",
                  xaxis_showgrid=True,
                  yaxis_showgrid=True,
                  width=1000,
                  height=600)
fmt_timestamps = [dt.strftime(prices['Time'][i], "%d-%b %H:%M") for i in range(len(prices))]
fig.add_trace(go.Scatter(x=prices['Time'], y=prices['Prices'],name="Price ($/MWh)", \
    line={'color':'#972f42'}, yaxis="y3"))
fig.add_trace(go.Scatter(x=result_load['time'], y=result_load['value'],name='Load (MW)', \
    line={'color':'#7F22A6'}))
fig.add_trace(go.Scatter(x=result_product['time'], y=result_product['value'],name='H2 Produced (kg)', \
    line={'color':'darkorange'}, yaxis="y2"))

for ser in fig['data']:
  ser['text'] = [dt.strftime(prices['Time'][i], "%d-%b %H:%M") for i in range(len(prices))]
  ser['hovertemplate'] = 'Time: %{text}<br>Value: %{y}'

fig.show()

```{include} example_electrolyser_msl.html 
```

## Specific Energy Consumption
The SEC functionality in NEMGLO is two-fold; a **fixed** profile, whereby all load (MW) produces an equivalent amount of hydrogen based on the defined SEC (kWh/kg), or a **variable** profile whereby the energy consumption (kWh/kg) varies depending on the load (MW).

To demonstrate these features, we iterate through an arbitrary simulation period and each iteration force the load to a certain MW value using the advanced feature `_set_force_h2_load`. These iterations are then repeated for each combination of `fixed` and `variable` with both `PEM` and `AE` electrolyser types.

In [237]:
inputdata = nemosis_data(intlength=30, local_cache=r'E:\TEMPCACHE')
start = "02/01/2020 00:00"
end = "02/01/2020 01:00"
region = 'VIC1'
inputdata.set_dates(start, end)
inputdata.set_region(region)
prices = inputdata.get_prices()

Compiling data for table DISPATCHPRICE.
Returning DISPATCHPRICE.


For a fixed SEC profile using PEM

In [ ]:
pem_xpoints_f, pem_ypoints_f = [], []

for x_val in range(20,101,1):
    P2G = Plan('P2G')
    P2G.load_market_prices(prices)

    h2e = Electrolyser(P2G, identifier='H2E')
    h2e.load_h2_parameters_preset(capacity = 100.0,
                              maxload = 100.0,
                              minload = 20.0,
                              offload = 0.0,
                              electrolyser_type = 'PEM',
                              sec_profile = 'fixed',
                              h2_price_kg = 6.0)
    h2e.add_electrolyser_operation()
    h2e._set_force_h2_load(mw_value=x_val)
    P2G.optimise()

    result_load = P2G.get_load()
    result_product = P2G.get_production()
    pem_xpoints_f += [float(result_load.loc[result_load['interval']==0,'value'])]
    pem_ypoints_f += [float(result_product.loc[result_product['interval']==0,'value'])]

For a variable SEC profile using PEM

In [ ]:
pem_xpoints, pem_ypoints = [], []

for x_val in range(20,101,1):
    P2G = Plan('P2G')
    P2G.load_market_prices(prices)

    h2e = Electrolyser(P2G, identifier='H2E')
    h2e.load_h2_parameters_preset(capacity = 100.0,
                              maxload = 100.0,
                              minload = 20.0,
                              offload = 0.0,
                              electrolyser_type = 'PEM',
                              sec_profile = 'variable',
                              h2_price_kg = 6.0)
    h2e.add_electrolyser_operation()
    h2e._set_force_h2_load(mw_value=x_val)
    P2G.optimise()

    result_load = P2G.get_load()
    result_product = P2G.get_production()
    pem_xpoints += [float(result_load.loc[result_load['interval']==0,'value'])]
    pem_ypoints += [float(result_product.loc[result_product['interval']==0,'value'])]

# Save predefined SEC points for plotting
pem_sec_spec = h2e._sec_variable_points

For a fixed profile using AE

In [ ]:
ae_xpoints_f, ae_ypoints_f = [], []

for x_val in range(20,101,1):
    P2G = Plan('P2G')
    P2G.load_market_prices(prices)

    h2e = Electrolyser(P2G, identifier='H2E')
    h2e.load_h2_parameters_preset(capacity = 100.0,
                              maxload = 100.0,
                              minload = 20.0,
                              offload = 0.0,
                              electrolyser_type = 'AE',
                              sec_profile = 'fixed',
                              h2_price_kg = 6.0)
    h2e.add_electrolyser_operation()
    h2e._set_force_h2_load(mw_value=x_val)
    P2G.optimise()

    result_load = P2G.get_load()
    result_product = P2G.get_production()
    ae_xpoints_f += [float(result_load.loc[result_load['interval']==0,'value'])]
    ae_ypoints_f += [float(result_product.loc[result_product['interval']==0,'value'])]

For a variable profile using AE

In [ ]:
ae_xpoints, ae_ypoints = [], []

for x_val in range(20,101,1):
    P2G = Plan('P2G')
    P2G.load_market_prices(prices)

    h2e = Electrolyser(P2G, identifier='H2E')
    h2e.load_h2_parameters_preset(capacity = 100.0,
                              maxload = 100.0,
                              minload = 20.0,
                              offload = 0.0,
                              electrolyser_type = 'AE',
                              sec_profile = 'variable',
                              h2_price_kg = 6.0)
    h2e.add_electrolyser_operation()
    h2e._set_force_h2_load(mw_value=x_val)
    P2G.optimise()

    result_load = P2G.get_load()
    result_product = P2G.get_production()
    ae_xpoints += [float(result_load.loc[result_load['interval']==0,'value'])]
    ae_ypoints += [float(result_product.loc[result_product['interval']==0,'value'])]

# Save predefined SEC points for plotting
ae_sec_spec = h2e._sec_variable_points

Note that for variable SEC profiles, NEMGLO uses a Special Ordered Set Type 2 which directly converts from load (MW) to hydrogen production (kg). As such there is no variable storing the SEC (kWh/kg) for each interval. We can infer this value by the calculation below.

In [254]:
# PEM
pem_sec_y_variable = [(pem_xpoints[i] * 0.5 * 1000) / pem_ypoints[i] \
                      for i in range(1,len(pem_xpoints))]
pem_sec_y_defined = [(pem_sec_spec['h2e_load'].to_list()[i] * 0.5 * 1000) \
                     / pem_sec_spec['h2_volume'].to_list()[i] for i in \
                     range(1,len(pem_sec_spec['h2e_load']))]
pem_sec_fix_y_variable = [(pem_xpoints_f[i] * 0.5 * 1000) / pem_ypoints_f[i] \
                          for i in range(1,len(pem_xpoints_f))]

## AE
ae_sec_y_variable = [(ae_xpoints[i] * 0.5 * 1000) / ae_ypoints[i] \
                     for i in range(1,len(ae_xpoints))]
ae_sec_y_defined = [(ae_sec_spec['h2e_load'].to_list()[i] * 0.5 * 1000) \
                    / ae_sec_spec['h2_volume'].to_list()[i] for i in \
                    range(1,len(ae_sec_spec['h2e_load']))]
ae_sec_fix_y_variable = [(ae_xpoints_f[i] * 0.5 * 1000) / ae_ypoints_f[i] \
                         for i in range(1,len(ae_xpoints_f))]

Plotting the relationship between the amount of hydrogen produced against load (MW) yields...

In [260]:
fig = go.Figure()
PALETTE = ['#b2d4ee','#849db1','#4f6980',
           '#B4E3BC','#89AE8F','#638b66',
           '#ffb04f','#de9945','#af7635',
           '#ff7371','#d6635f','#b65551',
           '#AD134C','#cc688d','#ff82b0']  

# PEM
fig.add_trace(go.Scatter(x=pem_sec_spec['h2e_load'][1:], y=pem_sec_spec['h2_volume'][1:], mode='markers', name="Input SEC points [PEM]",
    legendgroup='marker', marker_symbol="diamond", marker_size=14, line={'color':PALETTE[2]}))
fig.add_trace(go.Scatter(x=pem_xpoints, y=pem_ypoints, mode='lines', name="Variable SEC mode [PEM]", line_width=3,
    legendgroup='var', line={'color':PALETTE[2],'dash': 'dash'}))
fig.add_trace(go.Scatter(x=pem_xpoints_f, y=pem_ypoints_f, mode='lines', name="Fixed SEC mode [PEM]", line_width=3,
    legendgroup='fix', line={'color':PALETTE[2]}))

# AE
fig.add_trace(go.Scatter(x=ae_sec_spec['h2e_load'][1:], y=ae_sec_spec['h2_volume'][1:], mode='markers', name="Input SEC points [AE]",
    legendgroup='marker', marker_symbol="diamond", marker_size=14, line={'color':PALETTE[5]}))
fig.add_trace(go.Scatter(x=ae_xpoints, y=ae_ypoints, mode='lines', name="Variable SEC mode [AE]", line_width=3,
    legendgroup='var', line={'color':PALETTE[5],'dash': 'dash'}))
fig.add_trace(go.Scatter(x=ae_xpoints_f, y=ae_ypoints_f, mode='lines', name="Fixed SEC mode [AE]", line_width=3,
    legendgroup='fix',  line={'color':PALETTE[5]}))

# Layout
fig.update_layout(title="<b>NEMGLO Hydrogen Production vs Load Relationship</b>", titlefont=dict(size=24),
    margin=dict(l=20, r=20, t=50, b=0),
    xaxis=dict(title="Electrolyser Load (MW)", showgrid=True, mirror=True, titlefont=dict(size=24), tickfont=dict(size=24)),
    yaxis=dict(title="Hydrogen Produced (kg)", showgrid=True, mirror=True, titlefont=dict(size=24), tickfont=dict(size=24)),
    legend=dict(xanchor='center',x=0.5, y=-0.18, orientation='h', font=dict(size=20)),
    template="simple_white",
    width=1000,
    height=600,
    font_family="Times New Roman",
    )
fig.show()

```{note} The load region between <code>offload</code> and <code>minload</code> is omitted here since the hydrogen should not operate within that region. The MSL constraint shown prior enforces such behaviour.
```

```{include} example_electrolyser_h2produced.html 
```

Plotting the relationship between Specific Energy Consumption (SEC) and load (MW) yields...

In [261]:
# PEM
fig = go.Figure()
fig.add_trace(go.Scatter(x=pem_sec_spec['h2e_load'][1:], y=pem_sec_y_defined, mode='markers', name="Input SEC points [PEM]",
    legendgroup='marker', marker_symbol="diamond", marker_size=14, line={'color':PALETTE[2]}))
fig.add_trace(go.Scatter(x=pem_xpoints, y=pem_sec_y_variable, mode='lines', name="Variable SEC mode [PEM]", line_width=3,
    legendgroup='var', line={'color':PALETTE[2], 'dash': 'dash'}))
fig.add_trace(go.Scatter(x=pem_xpoints_f, y=pem_sec_fix_y_variable,  mode='lines', name="Fixed SEC mode [PEM]", line_width=3,
    legendgroup='fix', line={'color':PALETTE[2]}))

# AE
fig.add_trace(go.Scatter(x=ae_sec_spec['h2e_load'][1:], y=ae_sec_y_defined, mode='markers', name="Input SEC points [AE]",
    legendgroup='marker', marker_symbol="diamond", marker_size=14, line={'color':PALETTE[5]}))
fig.add_trace(go.Scatter(x=ae_xpoints, y=ae_sec_y_variable, mode='lines', name="Variable SEC mode [AE]", line_width=3,
    legendgroup='var', line={'color':PALETTE[5], 'dash': 'dash'}))
fig.add_trace(go.Scatter(x=ae_xpoints_f, y=ae_sec_fix_y_variable,  mode='lines', name="Fixed SEC mode [AE]", line_width=3,
    legendgroup='fix', line={'color':PALETTE[5]}))

# Layout
fig.update_layout(title="<b>NEMGLO Specific Energy Consumption vs Load Relationship</b>", titlefont=dict(size=24),
    margin=dict(l=20, r=20, t=50, b=0),
    xaxis=dict(title="Electrolyser Load (MW)", showgrid=True, mirror=True, titlefont=dict(size=24), tickfont=dict(size=24)),
    yaxis=dict(title="Specific Energy Consumption<br>(kWh/kg)", showgrid=True, mirror=True, titlefont=dict(size=24), tickfont=dict(size=24)),
    legend=dict(xanchor='center',x=0.5, y=-0.18, orientation='h', font=dict(size=20)),
    template="simple_white",
    width=1000,
    height=600,
    font_family="Times New Roman",
    )

fig.show()

```{include} example_electrolyser_sec.html 
```

## Hydrogen Storage

In [3]:
inputdata = nemosis_data(intlength=30, local_cache=r'E:\TEMPCACHE')
start = "02/01/2020 00:00"
end = "09/01/2020 00:00"
region = 'VIC1'
inputdata.set_dates(start, end)
inputdata.set_region(region)
prices = inputdata.get_prices()

Compiling data for table DISPATCHPRICE.
Returning DISPATCHPRICE.


In [11]:
P2G = Plan(identifier = "P2G")
P2G.load_market_prices(prices)

h2e = Electrolyser(P2G, identifier='H2E')
h2e.load_h2_parameters_preset(capacity = 100.0,
                            maxload = 100.0,
                            minload = 0.0,
                            offload = 0.0,
                            electrolyser_type = 'PEM',
                            sec_profile = 'fixed',
                            h2_price_kg = 6.0)
h2e.add_electrolyser_operation()

# Storage
h2e._storage_max = 50000000

h2e._set_h2_production_tracking()
h2e._set_h2_storage_max()

P2G.optimise()

result_load = P2G.get_load()


OPTIMISATION COMPLETE, Obj Value: -719478.052285357


In [12]:
P2G._out_vars['variable_name'].unique()

array(['H2E-mw_load', 'H2E-mw_load_sum', 'H2E-h2_produced',
       'H2E-h2_produced_sum', 'H2E-h2_stored'], dtype=object)

In [13]:
P2G._format_out_vars_timeseries('H2E-h2_stored')

,interval,value,time
0,0,757.575758,2020-01-02 00:30:00
1,1,1515.151515,2020-01-02 01:00:00
2,2,2272.727273,2020-01-02 01:30:00
3,3,3030.303030,2020-01-02 02:00:00
4,4,3787.878788,2020-01-02 02:30:00
...,...,...,...
331,331,248484.848485,2020-01-08 22:00:00
332,332,249242.424242,2020-01-08 22:30:00
333,333,250000.000000,2020-01-08 23:00:00
334,334,250757.575758,2020-01-08 23:30:00


In [14]:
P2G._format_out_vars_timeseries('H2E-h2_produced')

,interval,value,time
0,0,757.575758,2020-01-02 00:30:00
1,1,757.575758,2020-01-02 01:00:00
2,2,757.575758,2020-01-02 01:30:00
3,3,757.575758,2020-01-02 02:00:00
4,4,757.575758,2020-01-02 02:30:00
...,...,...,...
331,331,757.575758,2020-01-08 22:00:00
332,332,757.575758,2020-01-08 22:30:00
333,333,757.575758,2020-01-08 23:00:00
334,334,757.575758,2020-01-08 23:30:00
